<a href="https://colab.research.google.com/github/eumoas/Data-Engineer/blob/main/When_otherwise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

     |████████████████████████████████| 281.4 MB 34 kB/s 
     |████████████████████████████████| 198 kB 48.6 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=9041488b6e0fb88e500bace24c7539691c9a9dd1bbb84bd8c8145958515d2812
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkConf
import pyspark.sql.functions as F

In [ ]:
spark = (
    SparkSession.builder
                .master("local[3]")
                .appName("when_otherwise")
                .config("spark.ui.port", "4050")
                .getOrCreate()
)

In [ ]:
spark

In [ ]:
df = ( spark.read
            .format("csv")
            .option("header", "true")
            .option("inferschema", "true")
            .option("delimiter", ";")
            .load("/content/drive/MyDrive/Datasets/arquivo_geral.csv")
      
)

In [ ]:
df.show(20)

+------+------+----------+----------+---------------+-----------+----------------+
|regiao|estado|      data|casosNovos|casosAcumulados|obitosNovos|obitosAcumulados|
+------+------+----------+----------+---------------+-----------+----------------+
| Norte|    RO|2020-01-30|         0|              0|          0|               0|
| Norte|    RO|2020-01-31|         0|              0|          0|               0|
| Norte|    RO|2020-02-01|         0|              0|          0|               0|
| Norte|    RO|2020-02-02|         0|              0|          0|               0|
| Norte|    RO|2020-02-03|         0|              0|          0|               0|
| Norte|    RO|2020-02-04|         0|              0|          0|               0|
| Norte|    RO|2020-02-05|         0|              0|          0|               0|
| Norte|    RO|2020-02-06|         0|              0|          0|               0|
| Norte|    RO|2020-02-07|         0|              0|          0|               0|
| No

When - Otherwise

In [ ]:
#CRIANDO UMA NOVA COLUNA COM BASE EM DUAS CONDIÇÕES
df2 = (df.withColumn("Status", F.when(F.col("casosNovos") > 0, F.lit("Possui Casos Novos"))
                    .otherwise(F.lit("Não possui casos novos"))))

df2.show(1000)

In [ ]:
#CRIANDO UMA NOVA COLUNA COM BASE EM MAIS DE DUAS CONDIÇÕES
df3 = (df.withColumn("Status", F.when((F.col('casosNovos') >= 50) & (F.col('casosNovos') <= 100), F.lit("Entre 50 e 100"))
                                .when((F.col('casosNovos') > 100 ) & (F.col('casosNovos') <= 200), F.lit('Entre 101 e 200'))
                                .when((F.col('casosNovos') > 200 ) & (F.col('casosNovos') <= 300), F.lit('Entre 201 e 300'))
                                .otherwise(F.lit('Entre outra faixa de valores'))))

df3.show(20)

In [ ]:
#PERCORRENDO UMA SUBSTRING DENTRO DE UM DATAFRAME
df4 = df3.withColumn("Dia", F.lit(10)).withColumn("Mes", F.lit(1)).withColumn("Ano", F.lit(2020))

df4.select(F.col("regiao"), F.col("estado"), F.col("data"), F.col("Dia"), F.col("Mes"), F.col("Ano")).show(20)

# DIA F.substring(F.col("data"), 9, 2)
# MES F.substring(F.col("data"), 6, 2)
# ANO F.substring(F.col("data"), 1, 4)


+------+------+----------+---+---+----+
|regiao|estado|      data|Dia|Mes| Ano|
+------+------+----------+---+---+----+
| Norte|    RO|2020-01-30| 10|  1|2020|
| Norte|    RO|2020-01-31| 10|  1|2020|
| Norte|    RO|2020-02-01| 10|  1|2020|
| Norte|    RO|2020-02-02| 10|  1|2020|
| Norte|    RO|2020-02-03| 10|  1|2020|
| Norte|    RO|2020-02-04| 10|  1|2020|
| Norte|    RO|2020-02-05| 10|  1|2020|
| Norte|    RO|2020-02-06| 10|  1|2020|
| Norte|    RO|2020-02-07| 10|  1|2020|
| Norte|    RO|2020-02-08| 10|  1|2020|
| Norte|    RO|2020-02-09| 10|  1|2020|
| Norte|    RO|2020-02-10| 10|  1|2020|
| Norte|    RO|2020-02-11| 10|  1|2020|
| Norte|    RO|2020-02-12| 10|  1|2020|
| Norte|    RO|2020-02-13| 10|  1|2020|
| Norte|    RO|2020-02-14| 10|  1|2020|
| Norte|    RO|2020-02-15| 10|  1|2020|
| Norte|    RO|2020-02-16| 10|  1|2020|
| Norte|    RO|2020-02-17| 10|  1|2020|
| Norte|    RO|2020-02-18| 10|  1|2020|
+------+------+----------+---+---+----+
only showing top 20 rows



In [ ]:
#PERCORRENDO UMA SUBSTRING DENTRO DE UM DATAFRAME
df4 =( df3.withColumn("Dia", F.substring(F.col("data"), 9, 2))
          .withColumn("Mes", F.substring(F.col("data"), 6, 2))
          .withColumn("Ano", F.substring(F.col("data"), 1, 4))

)

# DIA F.substring(F.col("data"), 9, 2)
# MES F.substring(F.col("data"), 6, 2)
# ANO F.substring(F.col("data"), 1, 4)
df4.select(F.col("data"), F.col("Dia"), F.col("Mes"), F.col("Ano")).show(20)

+----------+---+---+----+
|      data|Dia|Mes| Ano|
+----------+---+---+----+
|2020-01-30| 30| 01|2020|
|2020-01-31| 31| 01|2020|
|2020-02-01| 01| 02|2020|
|2020-02-02| 02| 02|2020|
|2020-02-03| 03| 02|2020|
|2020-02-04| 04| 02|2020|
|2020-02-05| 05| 02|2020|
|2020-02-06| 06| 02|2020|
|2020-02-07| 07| 02|2020|
|2020-02-08| 08| 02|2020|
|2020-02-09| 09| 02|2020|
|2020-02-10| 10| 02|2020|
|2020-02-11| 11| 02|2020|
|2020-02-12| 12| 02|2020|
|2020-02-13| 13| 02|2020|
|2020-02-14| 14| 02|2020|
|2020-02-15| 15| 02|2020|
|2020-02-16| 16| 02|2020|
|2020-02-17| 17| 02|2020|
|2020-02-18| 18| 02|2020|
+----------+---+---+----+
only showing top 20 rows



In [ ]:
df5 =( df3.withColumn("Dia", F.substring(F.col("data"), 9, 2).cast("integer"))
          .withColumn("Mes", F.substring(F.col("data"), 6, 2).cast("integer"))
          .withColumn("Ano", F.substring(F.col("data"), 1, 4).cast("integer"))

)

df5.printSchema()

root
 |-- regiao: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- data: string (nullable = true)
 |-- casosNovos: integer (nullable = true)
 |-- casosAcumulados: integer (nullable = true)
 |-- obitosNovos: integer (nullable = true)
 |-- obitosAcumulados: integer (nullable = true)
 |-- Status: string (nullable = false)
 |-- Dia: integer (nullable = true)
 |-- Mes: integer (nullable = true)
 |-- Ano: integer (nullable = true)



In [ ]:
#FUNÇÕES PARA ORDENAÇÃO DE COLUNAS - ORDEM CRESCENTE OU DECRESCENTE SORT ou ORDERBY

#df5.show(10)

df5.select(F.col("estado"), F.col('casosNovos'), F.col("Dia"), F.col("Mes"), F.col("Ano")).sort(F.col("Dia").desc(), F.col("Mes")).show(200)
df5.select(F.col("estado"), F.col('casosNovos'), F.col("Dia"), F.col("Mes"), F.col("Ano")).orderBy(F.col("Mes").asc(), F.col("Dia")).show(200)

In [ ]:
#AGRUPAR OS RESULTADOS DE ACORDO COM UMA COLUNA ESPECÍFICA
#FAZER A SOMA DE CASOS NOVOS POR ESTADO

df5.groupBy(F.col("estado")).sum("casosNovos").max("casosNovos").show()

AttributeError: ignored

In [ ]:
#df5.show(5)

#QUANTIDADE DE ESTADOS POR REGIÃO NO DATAFRAME - COUNT() - countDistinct()
#df5.groupBy(F.col("regiao")).F.countDistinct(F.col("estado"))
df6 = df5.select(F.col("regiao"), F.col("estado")).distinct().groupBy("regiao")

df6.count().show()

+------------+-----+
|      regiao|count|
+------------+-----+
|    Nordeste|    9|
|         Sul|    3|
|     Sudeste|    4|
|Centro-Oeste|    4|
|       Norte|    7|
+------------+-----+



In [ ]:
df2 = df.select(F.col("regiao")).distinct().groupBy("regiao").count()
df2.show(20)

+------------+-----+
|      regiao|count|
+------------+-----+
|    Nordeste|  783|
|         Sul|  261|
|     Sudeste|  348|
|Centro-Oeste|  348|
|       Norte|  609|
+------------+-----+



In [ ]:
#FUNÇÃO DE AGREGAÇÃO - agg
df7 = ( df5.groupBy(F.col("estado"))
           .agg(F.sum("casosAcumulados"),
                F.max("casosNovos"),
                F.min("casosNovos"),
                F.avg("casosNovos")).show() 
)

#df5.printSchema()

+------+--------------------+---------------+---------------+------------------+
|estado|sum(casosAcumulados)|max(casosNovos)|min(casosNovos)|   avg(casosNovos)|
+------+--------------------+---------------+---------------+------------------+
|    SC|               19811|            192|              0| 13.89655172413793|
|    RO|                2314|             40|              0|3.7701149425287355|
|    PI|                2420|             41|              0| 3.413793103448276|
|    AM|               36838|            441|              0|  41.7816091954023|
|    RR|                3407|             41|              0|3.9655172413793105|
|    GO|                7121|             51|              0| 5.816091954022989|
|    TO|                 741|              7|              0|0.5747126436781609|
|    MT|                3386|             38|              0| 2.839080459770115|
|    SP|              266231|           2178|              0|229.93103448275863|
|    PB|                3840